In [1]:

from __future__ import print_function

import os
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose, UpSampling2D, Activation
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.layers.normalization import BatchNormalization

import numpy as np
import os
from skimage.io import imsave, imread

Using TensorFlow backend.


In [2]:
!pip install kaggle-cli

!kg download -u <kaggle-username> -p <password> -c ultrasound-nerve-segmentation -f 'train.zip'

    100% |████████████████████████████████| 71kB 2.7MB/s ta 0:00:01
    100% |████████████████████████████████| 5.3MB 273kB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 11.3MB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 11.7MB/s ta 0:00:01
  Running setup.py bdist_wheel for kaggle-cli ... done
  Stored in directory: /root/.cache/pip/wheels/16/28/c7/78658216416b7b481df91b485e43b6722daf59e71fc9cf9a58
  Running setup.py bdist_wheel for PrettyTable ... done
  Stored in directory: /root/.cache/pip/wheels/b6/90/7b/1c22b89217d0eba6d5f406e562365ebee804f0d4595b2bdbcd
  Running setup.py bdist_wheel for unicodecsv ... done
  Stored in directory: /root/.cache/pip/wheels/97/e2/16/219fa93b83edaff912b6805cfa19d0597e21f8d353f3e2d22f
  Running setup.py bdist_wheel for cmd2 ... done
  Stored in directory: /root/.cache/pip/wheels/aa/51/63/dfeef1471ea77d2e622021366ff563e07d5e95a502eb3eb19a
  Running setup.py bdist_wheel for pyperclip ... done
  Stored in directory: /roo

In [3]:
!unzip train.zip -d data

Archive:  train.zip
   creating: data/train/
  inflating: data/train/10_1.tif     
  inflating: data/train/10_10.tif    
  inflating: data/train/10_100.tif   
  inflating: data/train/10_100_mask.tif  
  inflating: data/train/10_101.tif   
  inflating: data/train/10_101_mask.tif  
  inflating: data/train/10_102.tif   
  inflating: data/train/10_102_mask.tif  
  inflating: data/train/10_103.tif   
  inflating: data/train/10_103_mask.tif  
  inflating: data/train/10_104.tif   
  inflating: data/train/10_104_mask.tif  
  inflating: data/train/10_105.tif   
  inflating: data/train/10_105_mask.tif  
  inflating: data/train/10_106.tif   
  inflating: data/train/10_106_mask.tif  
  inflating: data/train/10_107.tif   
  inflating: data/train/10_107_mask.tif  
  inflating: data/train/10_108.tif   
  inflating: data/train/10_108_mask.tif  
  inflating: data/train/10_109.tif   
  inflating: data/train/10_109_mask.tif  
  inflating: data/train/10_10_mask.tif  
  inflating: data/train/10_11.tif    


  inflating: data/train/10_9.tif     
  inflating: data/train/10_90.tif    
  inflating: data/train/10_90_mask.tif  
  inflating: data/train/10_91.tif    
  inflating: data/train/10_91_mask.tif  
  inflating: data/train/10_92.tif    
  inflating: data/train/10_92_mask.tif  
  inflating: data/train/10_93.tif    
  inflating: data/train/10_93_mask.tif  
  inflating: data/train/10_94.tif    
  inflating: data/train/10_94_mask.tif  
  inflating: data/train/10_95.tif    
  inflating: data/train/10_95_mask.tif  
  inflating: data/train/10_96.tif    
  inflating: data/train/10_96_mask.tif  
  inflating: data/train/10_97.tif    
  inflating: data/train/10_97_mask.tif  
  inflating: data/train/10_98.tif    
  inflating: data/train/10_98_mask.tif  
  inflating: data/train/10_99.tif    
  inflating: data/train/10_99_mask.tif  
  inflating: data/train/10_9_mask.tif  
  inflating: data/train/11_1.tif     
  inflating: data/train/11_10.tif    
  inflating: data/train/11_100.tif   
  inflating: data/

  inflating: data/train/11_85.tif    
  inflating: data/train/11_85_mask.tif  
  inflating: data/train/11_86.tif    
  inflating: data/train/11_86_mask.tif  
  inflating: data/train/11_87.tif    
  inflating: data/train/11_87_mask.tif  
  inflating: data/train/11_88.tif    
  inflating: data/train/11_88_mask.tif  
  inflating: data/train/11_89.tif    
  inflating: data/train/11_89_mask.tif  
  inflating: data/train/11_8_mask.tif  
  inflating: data/train/11_9.tif     
  inflating: data/train/11_90.tif    
  inflating: data/train/11_90_mask.tif  
  inflating: data/train/11_91.tif    
  inflating: data/train/11_91_mask.tif  
  inflating: data/train/11_92.tif    
  inflating: data/train/11_92_mask.tif  
  inflating: data/train/11_93.tif    
  inflating: data/train/11_93_mask.tif  
  inflating: data/train/11_94.tif    
  inflating: data/train/11_94_mask.tif  
  inflating: data/train/11_95.tif    
  inflating: data/train/11_95_mask.tif  
  inflating: data/train/11_96.tif    
  inflating: da

  inflating: data/train/12_91.tif    
  inflating: data/train/12_91_mask.tif  
  inflating: data/train/12_92.tif    
  inflating: data/train/12_92_mask.tif  
  inflating: data/train/12_93.tif    
  inflating: data/train/12_93_mask.tif  
  inflating: data/train/12_94.tif    
  inflating: data/train/12_94_mask.tif  
  inflating: data/train/12_95.tif    
  inflating: data/train/12_95_mask.tif  
  inflating: data/train/12_96.tif    
  inflating: data/train/12_96_mask.tif  
  inflating: data/train/12_97.tif    
  inflating: data/train/12_97_mask.tif  
  inflating: data/train/12_98.tif    
  inflating: data/train/12_98_mask.tif  
  inflating: data/train/12_99.tif    
  inflating: data/train/12_99_mask.tif  
  inflating: data/train/12_9_mask.tif  
  inflating: data/train/13_1.tif     
  inflating: data/train/13_10.tif    
  inflating: data/train/13_100.tif   
  inflating: data/train/13_100_mask.tif  
  inflating: data/train/13_101.tif   
  inflating: data/train/13_101_mask.tif  
  inflating: 

  inflating: data/train/13_84.tif    
  inflating: data/train/13_84_mask.tif  
  inflating: data/train/13_85.tif    
  inflating: data/train/13_85_mask.tif  
  inflating: data/train/13_86.tif    
  inflating: data/train/13_86_mask.tif  
  inflating: data/train/13_87.tif    
  inflating: data/train/13_87_mask.tif  
  inflating: data/train/13_88.tif    
  inflating: data/train/13_88_mask.tif  
  inflating: data/train/13_89.tif    
  inflating: data/train/13_89_mask.tif  
  inflating: data/train/13_8_mask.tif  
  inflating: data/train/13_9.tif     
  inflating: data/train/13_90.tif    
  inflating: data/train/13_90_mask.tif  
  inflating: data/train/13_91.tif    
  inflating: data/train/13_91_mask.tif  
  inflating: data/train/13_92.tif    
  inflating: data/train/13_92_mask.tif  
  inflating: data/train/13_93.tif    
  inflating: data/train/13_93_mask.tif  
  inflating: data/train/13_94.tif    
  inflating: data/train/13_94_mask.tif  
  inflating: data/train/13_95.tif    
  inflating: da

...

  inflating: data/train/6_24.tif     
  inflating: data/train/6_24_mask.tif  
  inflating: data/train/6_25.tif     
  inflating: data/train/6_25_mask.tif  
  inflating: data/train/6_26.tif     
  inflating: data/train/6_26_mask.tif  
  inflating: data/train/6_27.tif     
  inflating: data/train/6_27_mask.tif  
  inflating: data/train/6_28.tif     
  inflating: data/train/6_28_mask.tif  
  inflating: data/train/6_29.tif     
  inflating: data/train/6_29_mask.tif  
  inflating: data/train/6_2_mask.tif  
  inflating: data/train/6_3.tif      
  inflating: data/train/6_30.tif     
  inflating: data/train/6_30_mask.tif  
  inflating: data/train/6_31.tif     
  inflating: data/train/6_31_mask.tif  
  inflating: data/train/6_32.tif     
  inflating: data/train/6_32_mask.tif  
  inflating: data/train/6_33.tif     
  inflating: data/train/6_33_mask.tif  
  inflating: data/train/6_34.tif     
  inflating: data/train/6_34_mask.tif  
  inflating: data/train/6_35.tif     
  inflating: data/train/6_3

  inflating: data/train/7_15.tif     
  inflating: data/train/7_15_mask.tif  
  inflating: data/train/7_16.tif     
  inflating: data/train/7_16_mask.tif  
  inflating: data/train/7_17.tif     
  inflating: data/train/7_17_mask.tif  
  inflating: data/train/7_18.tif     
  inflating: data/train/7_18_mask.tif  
  inflating: data/train/7_19.tif     
  inflating: data/train/7_19_mask.tif  
  inflating: data/train/7_1_mask.tif  
  inflating: data/train/7_2.tif      
  inflating: data/train/7_20.tif     
  inflating: data/train/7_20_mask.tif  
  inflating: data/train/7_21.tif     
  inflating: data/train/7_21_mask.tif  
  inflating: data/train/7_22.tif     
  inflating: data/train/7_22_mask.tif  
  inflating: data/train/7_23.tif     
  inflating: data/train/7_23_mask.tif  
  inflating: data/train/7_24.tif     
  inflating: data/train/7_24_mask.tif  
  inflating: data/train/7_25.tif     
  inflating: data/train/7_25_mask.tif  
  inflating: data/train/7_26.tif     
  inflating: data/train/7_2

  inflating: data/train/8_116.tif    
  inflating: data/train/8_116_mask.tif  
  inflating: data/train/8_117.tif    
  inflating: data/train/8_117_mask.tif  
  inflating: data/train/8_118.tif    
  inflating: data/train/8_118_mask.tif  
  inflating: data/train/8_119.tif    
  inflating: data/train/8_119_mask.tif  
  inflating: data/train/8_11_mask.tif  
  inflating: data/train/8_12.tif     
  inflating: data/train/8_120.tif    
  inflating: data/train/8_120_mask.tif  
  inflating: data/train/8_12_mask.tif  
  inflating: data/train/8_13.tif     
  inflating: data/train/8_13_mask.tif  
  inflating: data/train/8_14.tif     
  inflating: data/train/8_14_mask.tif  
  inflating: data/train/8_15.tif     
  inflating: data/train/8_15_mask.tif  
  inflating: data/train/8_16.tif     
  inflating: data/train/8_16_mask.tif  
  inflating: data/train/8_17.tif     
  inflating: data/train/8_17_mask.tif  
  inflating: data/train/8_18.tif     
  inflating: data/train/8_18_mask.tif  
  inflating: data/t

  inflating: data/train/9_108.tif    
  inflating: data/train/9_108_mask.tif  
  inflating: data/train/9_109.tif    
  inflating: data/train/9_109_mask.tif  
  inflating: data/train/9_10_mask.tif  
  inflating: data/train/9_11.tif     
  inflating: data/train/9_110.tif    
  inflating: data/train/9_110_mask.tif  
  inflating: data/train/9_111.tif    
  inflating: data/train/9_111_mask.tif  
  inflating: data/train/9_112.tif    
  inflating: data/train/9_112_mask.tif  
  inflating: data/train/9_113.tif    
  inflating: data/train/9_113_mask.tif  
  inflating: data/train/9_114.tif    
  inflating: data/train/9_114_mask.tif  
  inflating: data/train/9_115.tif    
  inflating: data/train/9_115_mask.tif  
  inflating: data/train/9_116.tif    
  inflating: data/train/9_116_mask.tif  
  inflating: data/train/9_117.tif    
  inflating: data/train/9_117_mask.tif  
  inflating: data/train/9_118.tif    
  inflating: data/train/9_118_mask.tif  
  inflating: data/train/9_119.tif    
  inflating: da

  inflating: data/train/9_99_mask.tif  
  inflating: data/train/9_9_mask.tif  


In [4]:
data_path = 'data'

In [5]:
image_rows = 420
image_cols = 580
def create_train_data():
    train_data_path = os.path.join(data_path, 'train')
    images = os.listdir(train_data_path)
    total = len(images) // 2

    imgs = np.ndarray((total, image_rows, image_cols), dtype=np.uint8)
    imgs_mask = np.ndarray((total, image_rows, image_cols), dtype=np.uint8)

    i = 0
    print('Creating training images...\n')
    for image_name in images:
        if 'mask' in image_name:
            continue
        image_mask_name = image_name.split('.')[0] + '_mask.tif'
        img = imread(os.path.join(train_data_path, image_name), as_grey=True)
        img_mask = imread(os.path.join(train_data_path, image_mask_name), as_grey=True)

        img = np.array([img])
        img_mask = np.array([img_mask])

        imgs[i] = img
        imgs_mask[i] = img_mask

        if i % 100 == 0:
            print('Done: {0}/{1} images'.format(i, total))
        i += 1
    print('Loading done.')

    np.save('imgs_train.npy', imgs)
    np.save('imgs_mask_train.npy', imgs_mask)

In [6]:
create_train_data()

Creating training images...

Done: 0/5635 images
Done: 100/5635 images
Done: 200/5635 images
Done: 300/5635 images


/usr/local/lib/python2.7/site-packages/skimage/external/tifffile/tifffile.py:2611: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)


Done: 400/5635 images
Done: 500/5635 images
Done: 600/5635 images
Done: 700/5635 images
Done: 800/5635 images
Done: 900/5635 images
Done: 1000/5635 images
Done: 1100/5635 images
Done: 1200/5635 images
Done: 1300/5635 images
Done: 1400/5635 images
Done: 1500/5635 images
Done: 1600/5635 images
Done: 1700/5635 images
Done: 1800/5635 images
Done: 1900/5635 images
Done: 2000/5635 images
Done: 2100/5635 images
Done: 2200/5635 images
Done: 2300/5635 images
Done: 2400/5635 images
Done: 2500/5635 images
Done: 2600/5635 images
Done: 2700/5635 images
Done: 2800/5635 images
Done: 2900/5635 images
Done: 3000/5635 images
Done: 3100/5635 images
Done: 3200/5635 images
Done: 3300/5635 images
Done: 3400/5635 images
Done: 3500/5635 images
Done: 3600/5635 images
Done: 3700/5635 images
Done: 3800/5635 images
Done: 3900/5635 images
Done: 4000/5635 images
Done: 4100/5635 images
Done: 4200/5635 images
Done: 4300/5635 images
Done: 4400/5635 images
Done: 4500/5635 images
Done: 4600/5635 images
Done: 4700/5635 i

In [7]:
def load_train_data():
    imgs_train = np.load('imgs_train.npy')
    imgs_mask_train = np.load('imgs_mask_train.npy')
    return imgs_train, imgs_mask_train

img_rows = 96
img_cols = 96

smooth = 1.


def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [8]:

def get_unet():
    inputs = Input((img_rows, img_cols, 1))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3=Conv2D(128, kernel_size=(3,3),padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)    
    conv3=Conv2D(128, kernel_size=(3,3),padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3) 
    
    conv3=Conv2D(128, kernel_size=(3,3),padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)    
    conv3=Conv2D(128, kernel_size=(3,3),padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3) 

    up4 = concatenate([UpSampling2D(size= (2, 2))(conv3), conv2], axis=3)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up4)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)

    up5 = concatenate([UpSampling2D(size= (2, 2))(conv4), conv1], axis=3)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up5)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)

    conv6 = Conv2D(1, (1, 1), activation='sigmoid')(conv5)

    model = Model(inputs=[inputs], outputs=[conv6])

    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

    return model


In [9]:
def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True)

    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p

In [10]:
from sklearn.model_selection import train_test_split

def train_and_predict():
    print('-'*30)
    print('Loading and preprocessing train data...')
    print('-'*30)
    imgs_train, imgs_mask_train = load_train_data()

    imgs_train = preprocess(imgs_train)
    imgs_mask_train = preprocess(imgs_mask_train)

    imgs_train = imgs_train.astype('float32')
    mean = np.mean(imgs_train)  # mean for data centering
    std = np.std(imgs_train)  # std for data normalization

    imgs_train -= mean
    imgs_train /= std

    imgs_mask_train = imgs_mask_train.astype('float32')
    imgs_mask_train /= 255.  # scale masks to [0, 1]

    print('-'*30)
    print('Creating and compiling model...')
    print('-'*30)
    model = get_unet()
    #model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)
    X_train, X_valid, Y_train, Y_valid=train_test_split(imgs_train,imgs_mask_train,test_size=0.3, random_state=42)

    print('-'*30)
    print('Fitting model...')
    print('-'*30)
    model.fit(X_train,Y_train, batch_size=32, nb_epoch=100, verbose=1, shuffle=True,
              validation_data=(X_valid,Y_valid))
    score = model.evaluate(X_valid,Y_valid, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

In [11]:
if __name__ == '__main__':
    train_and_predict()

------------------------------
Loading and preprocessing train data...
------------------------------


/usr/local/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


------------------------------
Creating and compiling model...
------------------------------
------------------------------
Fitting model...
------------------------------


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:33: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 3944 samples, validate on 1691 samples
Epoch 1/100
3944/3944 [==============================] - 36s - loss: -0.0462 - dice_coef: 0.0462 - val_loss: -0.0254 - val_dice_coef: 0.0254
Epoch 2/100
3944/3944 [==============================] - 33s - loss: -0.2825 - dice_coef: 0.2825 - val_loss: -0.0195 - val_dice_coef: 0.0195
Epoch 3/100
3944/3944 [==============================] - 34s - loss: -0.3911 - dice_coef: 0.3911 - val_loss: -0.2153 - val_dice_coef: 0.2153
Epoch 4/100
3944/3944 [==============================] - 34s - loss: -0.4324 - dice_coef: 0.4324 - val_loss: -0.3776 - val_dice_coef: 0.3776
Epoch 5/100
3944/3944 [==============================] - 34s - loss: -0.4588 - dice_coef: 0.4588 - val_loss: -0.4325 - val_dice_coef: 0.4325
Epoch 6/100
3944/3944 [==============================] - 34s - loss: -0.4779 - dice_coef: 0.4779 - val_loss: -0.4586 - val_dice_coef: 0.4586
Epoch 7/100
3944/3944 [==============================] - 34s - loss: -0.4928 - dice_coef: 0.4928 - val_los